In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchaudio
import pandas as pd
import os
from tqdm import tqdm
from glob import glob

import torch.jit as jit
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.models import get_model

import openvino as ov
import scipy
from functools import partial

/home/cedric/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class Config:
    duration = 10
    sample_rate = 32000
    target_length = 384
    n_mels = 128
    n_fft = 2028
    window = 2028
    audio_len = duration*sample_rate
    hop_length = audio_len // (target_length-1)
    fmin = 20
    fmax = 16000
    top_db = 80

    n_classes = 182
    model_name = 'efficientnet_v2_s'
    checkpoint_dir = 'checkpoints/2024-05-07_08-48-51_fold-0_dim-128x384_model-efficientnet_v2_s'

    standardize = False
    dataset_mean = [-16.8828]
    dataset_std = [12.4019]

    loss = 'bce'
    use_openvino = True

In [15]:
def create_frames(waveform, duration=5, sr=32000):
    frame_size = int(duration * sr)
    surplus = waveform.size(-1)%frame_size
    if surplus > 0:
        waveform = waveform[:, :-surplus]
    frames = waveform.view(-1, 1, frame_size)
    return frames

class AudioDatasetInference(Dataset):
    def __init__(
            self, 
            files,
            targets = None, 
            n_classes = 182,
            duration = 5,
            sample_rate = 32000,
            target_length = 384,
            n_mels = 128,
            n_fft = 2028,
            window = 2028,
            hop_length = None,
            fmin = 20,
            fmax = 16000,
            top_db = 80,
            standardize=True,
            mean=None,
            std=None
            ):
        super(AudioDatasetInference, self).__init__()
        self.files = files
        self.targets = targets
        self.n_classes = n_classes
        self.duration = duration
        self.sample_rate = sample_rate
        self.audio_len = duration*sample_rate
        self.target_length = target_length
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.window = window
        self.hop_length = self.audio_len // (target_length-1) if not hop_length else hop_length
        self.fmin = fmin
        self.fmax = fmax
        self.top_db = top_db
        self.standardize = standardize

        self.to_mel_spectrogramn = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(self.sample_rate, n_fft=self.n_fft, win_length=self.window,  
                                                 hop_length=self.hop_length, n_mels=self.n_mels, 
                                                 f_min=self.fmin, f_max=self.fmax),
            torchaudio.transforms.AmplitudeToDB(top_db=self.top_db)
        )
        if mean is not None:
            self.to_mel_spectrogramn.append(v2.Normalize(mean=mean, std=std))

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        if self.targets is not None:
            label = torch.tensor(self.targets[idx])

        file = self.files[idx]
        waveform, sr = torchaudio.load(file)
        frames = create_frames(waveform)
        spec = self.to_mel_spectrogramn(frames)
        # Standardize
        if self.standardize:
            spec = (spec - spec.mean()) / spec.std()

        # expand to 3 channels for imagenet trained models
        spec = spec.expand(-1,3,-1,-1)

        if self.targets is not None:
            return spec, label
        else:
            return spec, file

In [16]:
class BasicClassifier(jit.ScriptModule):
    def __init__(self, n_classes, model_name, pretrained=True):
        super(BasicClassifier, self).__init__()
        weights = 'DEFAULT' if pretrained else None
        self.backbone = get_model(model_name, weights=weights).features
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(0.2, inplace=True),
            nn.Linear(1280, n_classes)
            )
        
    @jit.script_method
    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x).squeeze(dim=(-1,-2))
        x = self.classifier(x)
        return x

In [17]:
base_dir = 'data'
train_dir = base_dir + '/train_audio/'
test_dir = base_dir + '/test_soundscapes/'
unlabeled_dir = base_dir + '/unlabeled_soundscapes/'

class_names = sorted(os.listdir(train_dir))
n_classes = len(class_names)
class_labels = list(range(n_classes))
label2name = dict(zip(class_labels, class_names))
name2label = {v:k for k,v in label2name.items()}

In [18]:
test_paths = glob(base_dir + '/test_soundscapes/*ogg')
if len(test_paths)==0:
    test_paths = glob(base_dir + '/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,data/unlabeled_soundscapes/2003908684.ogg
1,data/unlabeled_soundscapes/514918048.ogg
2,data/unlabeled_soundscapes/62932451.ogg
3,data/unlabeled_soundscapes/735779963.ogg
4,data/unlabeled_soundscapes/1097804416.ogg


In [19]:
test_dataset = AudioDatasetInference(
    test_df['filepath'].values, 
    targets=None, 
    n_classes=Config.n_classes,
    duration=5,
    sample_rate=Config.sample_rate,
    target_length=Config.target_length,
    n_mels=Config.n_mels,
    n_fft=Config.n_fft,
    window=Config.window,
    hop_length=Config.hop_length,
    fmin=Config.fmin,
    fmax=Config.fmax,
    top_db=Config.top_db,
    standardize=Config.standardize,
    mean=Config.dataset_mean,
    std=Config.dataset_std
    )

In [20]:
device = torch.device('cpu')

model = BasicClassifier(Config.n_classes, pretrained=False, model_name=Config.model_name).to(device)
checkpoint_name = Config.checkpoint_dir + '/checkpoint.pth'
checkpoint_ov = Config.checkpoint_dir + '/checkpoint.xml'

if Config.use_openvino:
    core = ov.Core()
    ov_model = core.read_model(checkpoint_ov)
    model = ov.compile_model(ov_model)
else:
    checkpoint = torch.load(checkpoint_name, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    model = torch.jit.optimize_for_inference(torch.jit.script(model.eval()))

In [21]:
if Config.use_openvino:
    ids = []
    preds = np.empty(shape=(0, n_classes), dtype='float32')
    output_layer = model.output(0)
    if Config.loss == 'crossentropy':
        final_activation = partial(scipy.special.softmax, axis=1)
    elif Config.loss == 'bce':
        final_activation = scipy.special.expit

    test_iter = tqdm(range(len(test_dataset)))
    for i in test_iter:
        specs, file = test_dataset[i]
        filename = file.split('/')[-1][:-4]
        specs = specs.to(device)

        outs = model([specs])[output_layer]
        outs = final_activation(outs)

        frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(specs))]
        ids += frame_ids

        preds = np.concatenate([preds, outs], axis=0)

100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


In [22]:
preds[0]

array([0.01354243, 0.00463232, 0.01019922, 0.00372471, 0.01558469,
       0.00531036, 0.00400648, 0.00861809, 0.00664017, 0.03108814,
       0.02604451, 0.00368659, 0.00469115, 0.00825123, 0.0043385 ,
       0.00447994, 0.00411917, 0.01371466, 0.00652057, 0.01450375,
       0.03388326, 0.00564398, 0.00430619, 0.00341961, 0.00485846,
       0.00532798, 0.0063318 , 0.00945263, 0.00519515, 0.00496764,
       0.00687671, 0.00412886, 0.0095806 , 0.00723024, 0.00339576,
       0.00492607, 0.00474458, 0.01465122, 0.00790396, 0.15657096,
       0.02197172, 0.00735473, 0.00493682, 0.02360979, 0.12066981,
       0.00626087, 0.00639833, 0.01020818, 0.00781487, 0.00498089,
       0.00948554, 0.00452374, 0.00533505, 0.09196817, 0.00462226,
       0.00893404, 0.00372816, 0.02204976, 0.00812229, 0.00801881,
       0.00553835, 0.00364513, 0.01307441, 0.00440984, 0.00677614,
       0.01120832, 0.00528099, 0.00402772, 0.00526967, 0.00418823,
       0.07891361, 0.05529175, 0.02128759, 0.0073865 , 0.00335

In [12]:
if not Config.use_openvino:
    ids = []
    preds = np.empty(shape=(0, n_classes), dtype='float32')

    test_iter = tqdm(range(len(test_dataset)))
    for i in test_iter:
        specs, file = test_dataset[i]
        filename = file.split('/')[-1][:-4]
        specs = specs.to(device)

        with torch.no_grad():
            outs = model(specs)
            if Config.loss == 'crossentropy':
                outs = nn.functional.softmax(outs, dim=1).detach().cpu().numpy()
            elif Config.loss == 'bce':
                outs = outs.sigmoid().detach().cpu().numpy()

        frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(specs))]
        ids += frame_ids

        preds = np.concatenate([preds, outs], axis=0)

100%|██████████| 10/10 [03:01<00:00, 18.11s/it]


In [13]:
preds[0]

array([0.01350329, 0.00460619, 0.01018148, 0.00371789, 0.01549886,
       0.00530701, 0.00399118, 0.00860376, 0.00662451, 0.03112487,
       0.02614272, 0.00366814, 0.00467686, 0.00825858, 0.00437972,
       0.00446059, 0.00410759, 0.01369147, 0.00647925, 0.01442849,
       0.03371698, 0.0056321 , 0.00429275, 0.00341051, 0.00482742,
       0.00531856, 0.00631539, 0.00944804, 0.00516517, 0.00495279,
       0.00686402, 0.00411849, 0.00954771, 0.00722964, 0.0033883 ,
       0.00490069, 0.0047326 , 0.01472271, 0.00785903, 0.15679432,
       0.02211216, 0.00733071, 0.00492099, 0.02350239, 0.12148808,
       0.00621762, 0.00637341, 0.01017278, 0.00779372, 0.00496722,
       0.00945295, 0.00450854, 0.00532708, 0.09211251, 0.00460476,
       0.00891368, 0.00371208, 0.02223352, 0.00809187, 0.00803127,
       0.005537  , 0.00362622, 0.01311851, 0.00440125, 0.0067511 ,
       0.01121161, 0.00526754, 0.00401598, 0.0052518 , 0.00416266,
       0.07861832, 0.05551109, 0.02121472, 0.00733573, 0.00333

In [18]:
# Submit prediction
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, class_names] = preds
pred_df.to_csv('submission.csv',index=False)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,XC756601_5,0.005847,0.011639,0.002512,0.000110,0.019070,0.000015,0.000609,0.000143,0.000069,...,0.000811,0.000281,0.001206,0.000926,0.005042,0.010253,0.000052,0.000306,0.000032,0.004853
1,XC756601_10,0.008243,0.007165,0.001763,0.000092,0.018658,0.000011,0.000512,0.000136,0.000104,...,0.000528,0.000333,0.002916,0.001131,0.004920,0.010568,0.000061,0.000224,0.000028,0.005552
2,XC756601_15,0.010177,0.009778,0.003066,0.000166,0.017798,0.000020,0.000657,0.000119,0.000143,...,0.000895,0.000277,0.001253,0.001104,0.007026,0.008240,0.000074,0.000242,0.000039,0.006269
3,XC756601_20,0.006208,0.006755,0.002717,0.000150,0.014288,0.000024,0.000561,0.000175,0.000171,...,0.000782,0.000253,0.001888,0.001559,0.007132,0.009607,0.000056,0.000403,0.000023,0.012148
4,XC756601_25,0.006516,0.006837,0.001713,0.000164,0.020643,0.000016,0.000494,0.000309,0.000087,...,0.000877,0.000263,0.001813,0.001479,0.005192,0.010191,0.000060,0.000307,0.000018,0.007364
